In [1]:
!pip install delu
!pip install ucimlrepo
!pip install gdown
!pip install rtdl_num_embeddings
!git clone https://github.com/gbulgakov/testing-kan.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [2]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import optuna
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
from typing import Literal, Optional
from torch import Tensor
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import rtdl_num_embeddings
import delu
from IPython.display import FileLink
from tqdm import tqdm
import wandb
wandb.login(key='')

# наши импорты
sys.path.append('/kaggle/working/testing-kan/')
from optimizers.ademamix import AdEMAMix
from models.efficient_kan import KAN
from models.fastkan import FastKAN
from utils import utils
from models.prepare_model import model_init_preparation, ModelWithEmbedding, MLP
from utils.tg_bot import send_telegram_file, send_telegram_message

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gbulgakov (georgy-bulgakov) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
# !gdown 1xvRa_-OEeG6xNRYE5V5iAfTwyWM1NiLl # otto
# !gdown 1tYyhbHdYs_8I9jvXznMoeUAfBzwitaax # house
# !gdown 1hy1dOAL2SE-XZSuMcjLcVgml2CoYkF9q # higgs-small
!gdown 1hr076cK9QFxH6YZRg5V4av-H7IAve59r # gesture
# !gdown 1ZNScy5fgqtgudT6MZ4EjLt1nwdqirtmX # fb-comments
!gdown 1s0w7gnhiwBCkF49Wdi_cUDpUtXlz2_6q # eye
#!gdown 1T04iP04UGVo95Om84ww1Ed8AFNziOaeY # covtype
!gdown 1GOkNlinj4zHVSNKbqjN1rR4cvsAf2IgR # churn
!gdown 11B-l4EasJkclK_Q-RBcxYfGJLSvz-v5c # california
!gdown 1p8uqDPMfRlFIc69m7iikS6wGkA6JGj1H # adult

Downloading...
From: https://drive.google.com/uc?id=1s0w7gnhiwBCkF49Wdi_cUDpUtXlz2_6q
To: /kaggle/working/eye.zip
100%|█████████████████████████████████████████| 534k/534k [00:00<00:00, 109MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GOkNlinj4zHVSNKbqjN1rR4cvsAf2IgR
To: /kaggle/working/churn.zip
100%|█████████████████████████████████████████| 453k/453k [00:00<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=11B-l4EasJkclK_Q-RBcxYfGJLSvz-v5c
To: /kaggle/working/california.zip
100%|████████████████████████████████████████| 591k/591k [00:00<00:00, 98.3MB/s]


In [7]:
DATASETS = ['adult', 'california', 'churn', 'covtype', 'eye', 'fb-comments',
           'gesture', 'higgs-small', 'house', 'microsoft', 'otto', 'santander']


BATCH_SIZES = {'gesture' : 128, 'churn' : 128, 'california' : 256, 'house' : 256, 'adult' : 256, 'otto' : 512, 
               'higgs-small' : 512, 'fb-comments' : 512, 'santander' : 1024, 'covtype' : 1024, 'microsoft' : 1024, 'eye': 128}

REGRESSION = ['house', 'fb-comments', 'microsoft', 'california']
MULTICLASS = ['covtype', 'eye', 'gesture', 'otto']
BINCLASS = ['adult', 'churn', 'higgs-small', 'santander']

OPTIMIZERS = {'adamw' : torch.optim.AdamW,
              'ademamix' : AdEMAMix
             }

In [8]:
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import ExponentialLR, StepLR, CosineAnnealingLR
from torch.nn import MSELoss
import torch.nn as nn
import time


def apply_model(batch: dict[str, Tensor], model) -> Tensor:
    return model(batch['X_num'], batch.get('X_cat')).squeeze(-1)

def train_epoch(model, device, dataset, loss_fn, optimizer, scheduler):

    dataset_name = dataset['info']['id'].split('--')[0]
    task_type = dataset['info']['task_type']
    batch_size = BATCH_SIZES[dataset_name]

    model.to(device)
    model.train()
    train_loss = 0.0
    pred = []
    gt = [] # настоящие таргеты
    start_time = time.time()

    for data in delu.iter_batches(dataset['train'], shuffle=True, batch_size=batch_size):
        for key, tensor in data.items():
            data[key] = tensor.to(device)
        # обучение
        optimizer.zero_grad()
        output = apply_model(data, model)
        if task_type == 'multiclass':
            data['y'] = data['y'].long()
        loss_value = loss_fn(output, data['y']) 
        loss_value.backward()
        optimizer.step()
        # сохранение истории
        train_loss += loss_value.item()
        if output.dim() > 1:
            pred.append(output.argmax(1))
        else:
            pred.append(output >= 0.5)
        gt.append(data['y'])
    scheduler.step()

    end_time = time.time()
    epoch_time = end_time - start_time

    num_batches = dataset['train']['y'].shape[0] // batch_size + 1
    pred = torch.cat(pred)
    gt = torch.cat(gt)
    train_accuracy = (pred == gt).float().mean().item()

    return train_loss / num_batches, train_accuracy, epoch_time # с нормировкой
    
def validate(model, device, dataset, loss_fn, part='val'):
    model.eval()
    model.to(device)
    val_loss = 0.0

    pred = []
    gt = [] # настоящие таргеты

    dataset_name = dataset['info']['id'].split('--')[0]
    task_type = dataset['info']['task_type']
    batch_size = BATCH_SIZES[dataset_name]

    with torch.no_grad():
        start_time = time.time()
        for data in delu.iter_batches(dataset[part], shuffle=False, batch_size=batch_size):
            for key, tensor in data.items():
                data[key] = tensor.to(device)
            output = apply_model(data, model)
            if task_type == 'multiclass':
                data['y'] = data['y'].long()
            val_loss += loss_fn(output, data['y']).item()
            if output.dim() > 1:
                pred.append(output.argmax(1))
            else:
                pred.append(output >= 0.5)
            gt.append(data['y'])
        end_time = time.time()
        val_time = end_time - start_time
        

    num_batches = dataset[part]['y'].shape[0] // batch_size + 1
    pred = torch.cat(pred)
    gt = torch.cat(gt)
    val_accuracy = (pred == gt).float().mean().item()

    return val_loss / num_batches, val_accuracy, val_time # с нормировкой

Возможно ``model.to()`` лучше вызывать 1 раз.

In [20]:
def train(
    epochs, model, model_name,
    device, dataset, loss_fn,
    optimizer
):
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    model.to(device)
    dataset_name = dataset['info']['id'].split('--')[0]
    task_type = dataset['info']['task_type']
    batch_size = BATCH_SIZES[dataset_name]

    train_times = []
    val_times = []
    for epoch in tqdm(range(epochs), desc = f'{model_name} on {dataset_name}'):
        train_loss, train_acc, train_time = train_epoch(model, device, dataset, loss_fn, optimizer, scheduler)
        val_loss, val_acc, val_time = validate(model, device, dataset, loss_fn)

        wandb.log({
            'epoch' : epoch,
            'train_loss' : train_loss,
            'train_acc' : train_acc,
            'val_loss' : val_loss,
            'val_acc' : val_acc,
            'lr' : scheduler.get_last_lr()[0]
        })
        train_times.append(train_time)
        val_times.append(val_time)

    # размерность входа backbone
    in_features = dataset['train']['X_num'].shape[1]  # Количество числовых признаков
    if 'X_cat' in dataset['train']:
        in_features += dataset['train']['X_cat'].shape[1]  # Добавляем категориальные признаки

    
    wandb.log({
        'train_epoch_time' : sum(train_times) / epochs,
        'val_epoch_time' : sum(val_times) / epochs,
        'num_params' : utils.count_parameters(model),
        'in_features' : in_features,
        'out_features' : (1 if task_type != 'multiclass' else dataset['info']['n_classes'])
        # ширины и так будут залоггированы
    })

In [10]:
def get_optimizer(optim_name, model_params, config):
    optim_class = OPTIMIZERS[optim_name]
    optim_kwargs = {'lr' : config['lr']}
    if optim_name != 'muon': # это на будущее
        optim_kwargs['weight_decay'] = config['weight_decay']
    return optim_class(model_params, **optim_kwargs)

Функция тюнинга.

In [21]:
def wandb_tuning(project_name, dataset_name, 
                 model_name, emb_name, optim_name, 
                 dataset, num_epochs=10, num_trials=70):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset_info = dataset['info']
    num_cont_cols = dataset['train']['X_num'].shape[1]
    sweep_name = f'tuning {model_name}_{emb_name}_{optim_name} on {dataset_name}'

    # просто оборачиваем нашу train
    def sweep_wrapper():
        with wandb.init(
            project=f'{project_name}',
            group=f'dataset_{dataset_name}',
            tags=[f'model_{model_name}', f'emb_{emb_name}', f'optim_{optim_name}', f'dataset_{dataset_name}', 'tuning'],
            config=sweep_config
        ) as run:
            config = wandb.config
            _, backbone, bins, loss_fn = model_init_preparation(
                config=config,
                dataset=dataset,
                model_name=model_name,
                emb_name=emb_name
            )
            model = ModelWithEmbedding(
                n_cont_features=num_cont_cols,
                d_embedding=config.get('d_embedding', None),
                emb_name=emb_name,
                backbone_model=backbone,
                bins=bins, 
                sigma=config.get('sigma', None)
            )
            train(
                epochs=num_epochs,
                model=model,
                model_name=f'{model_name}_{emb_name}_{optim_name}',
                device=device,
                dataset=dataset,
                loss_fn=loss_fn,
                optimizer=get_optimizer(optim_name, model.parameters(), config),
            )
    sweep_config = utils.get_sweep_config(model_name, emb_name, dataset_info['task_type'], 
                                    f'tuning {model_name}_{emb_name}_{optim_name} on {dataset_name}')
    
    sweep_id = wandb.sweep(sweep=sweep_config,
                           project=f'{project_name}',
                           entity='georgy-bulgakov') 
    wandb.agent(sweep_id, sweep_wrapper, count=num_trials)
    return sweep_id

Функция для тестирования на разных сидах.

In [24]:
def test_best_model(best_params, project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs=10):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_cont_cols = dataset['train']['X_num'].shape[1]
    d_embedding = best_params.get('d_embedding', None)
    sigma = best_params.get('sigma', None)

    # подготовка логирования
    test_accuracies = []
    test_losses = []
    test_times = []
    train_times = []

    testing_config = utils.get_test_config(dataset['info']['task_type'], 
                                     f'testing {model_name}_{emb_name}_{optim_name} on {dataset_name}')
    # обертка тестирования
    def test_wrapper():
        with wandb.init(
            project=f'{project_name}',
            group=f'dataset_{dataset_name}',
            tags=[f'model_{model_name}', f'emb_{emb_name}', f'optim_{optim_name}', f'dataset_{dataset_name}', 'testing'],
            config=testing_config
        ) as run:
            config = wandb.config
            # seed + подготовка модели
            utils.seed_everything(config['seed'])
            layers, backbone, bins, loss_fn = model_init_preparation(
                config=best_params,
                dataset=dataset,
                model_name=model_name,
                emb_name=emb_name
            )
            model = ModelWithEmbedding(
                n_cont_features=num_cont_cols,
                d_embedding=d_embedding,
                emb_name=emb_name,
                backbone_model=backbone,
                bins=bins, 
                sigma=sigma
            )

            start_time = time.time()
            train(
                epochs=num_epochs,
                model=model,
                model_name=f'{model_name}_{emb_name}_{optim_name}',
                device=device,
                dataset=dataset,
                loss_fn=loss_fn,
                optimizer=get_optimizer(optim_name, model.parameters(), best_params)
            )
            end_time = time.time()
            test_loss, test_acc, test_time = validate(model, device, dataset, loss_fn, 'test')
            # Логируем результаты для каждого сида
            run.log({
                'test_loss': test_loss,
                'test_acc': test_acc,
                'full_train_time': end_time - start_time,
                'test_time': test_time,
                'seed': config['seed']
            })

            test_accuracies.append(test_acc)
            test_losses.append(np.sqrt(test_loss)) # переходим к RMSE
            test_times.append(test_time)
            train_times.append(end_time - start_time)

    test_id = wandb.sweep(sweep=testing_config,
                           project=f'{project_name}',
                           entity='georgy-bulgakov') 
    wandb.agent(test_id, test_wrapper)

    # Создаем финальный run для агрегированных результатов
    with wandb.init(
        project=f"{project_name}",
        group=f'dataset_{dataset_name}',
        name="aggregated_results",
        tags=[f'model_{model_name}', f'emb_{emb_name}', f'optim_{optim_name}', f'dataset_{dataset_name}', 'testing'],
        config=best_params
    ) as run:
        stats = {
            'model' : f'{model_name}_{emb_name}_{optim_name}',
            'mean_test_acc': np.mean(test_accuracies),
            'std_test_acc': np.std(test_accuracies),
            'mean_test_loss': np.mean(test_losses),
            'std_test_loss': np.std(test_losses),
            'mean_test_time': np.mean(test_times),
            'mean_train_time': np.mean(train_times),
            'all_test_accs': test_accuracies,
            'all_test_losses': test_losses,
            'all_test_times': test_times,
            'all_train_times': train_times
        }
        
        run.log(stats)
        stats['name'] = f'{model_name}_{emb_name}_{optim_name}'
        
    keys = ['model', 'mean_test_acc', 'std_test_acc', 'mean_test_loss', 'std_test_loss', 'mean_test_time', 'mean_train_time']
    return {key : stats[key] for key in keys} # чисто технически для удобства

In [17]:
from IPython.display import clear_output
def run_single_model(project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs, num_trials):
    sweep_id = wandb_tuning(project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs, num_trials)
    clear_output(wait=True)
    # вспоминаем лучшие параметры
    api = wandb.Api()
    sweep = api.sweep(f'georgy-bulgakov/{project_name}/{sweep_id}')
    runs = sweep.runs
    best_run = None
    if dataset['info']['task_type'] == 'regression':
        best_run = min(runs, key=lambda run : run.summary.get('val_loss'))
    else:
        best_run = max(runs, key=lambda run : run.summary.get('val_acc'))
    best_params = best_run.config
    
    stats = test_best_model(best_params, project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs)
    return stats, best_params

In [16]:
import pandas as pd
import pickle
def run_single_dataset(project_name, dataset_name, 
                       optim_names, emb_names, model_names, 
                       num_epochs, num_trials):
    # dataset_type = dataset_info['type']
    dataset = utils.load_dataset(dataset_name)
    results = []
    pkl_logs = {}

    for model_name in model_names: # можно оставить только kan, тогда model_names = ['kan']
        for optim_name in optim_names:
            for emb_name in emb_names:
                stats, best_params = run_single_model(project_name, dataset_name, model_name, emb_name, optim_name, dataset, num_epochs, num_trials)
                results.append(stats)
                clear_output(wait=True)
                pkl_logs[f'{model_name}_{emb_name}_{optim_name}'] = (stats | best_params)
                send_telegram_message(f'✅ {model_name}_{emb_name}_{optim_name} finished on {dataset_name}')

    
    with wandb.init(
        project=project_name,
        group=f'dataset_{dataset_name}',
        name='models_comparison'
    ) as run:
        run.log({
            f'final_table_{dataset_name}' : wandb.Table(dataframe=pd.DataFrame(results))
        })

    with open(f'/kaggle/working/{dataset_name}.pkl', 'wb') as f:
        pickle.dump(pkl_logs, f)

    send_telegram_file(f'{dataset_name}.pkl')
    return results



In [26]:
optim_names = ['ademamix']
model_names = ['fast_kan']
emb_names = ['none', 'periodic']
project_name = 'TAB-KAN study'

for dataset_name in ['churn', 'california']:
    run_single_dataset(project_name, dataset_name, optim_names, emb_names, model_names, 10, 50)

Create sweep with ID: q1jdy35i
Sweep URL: https://wandb.ai/georgy-bulgakov/TAB-KAN%20study/sweeps/q1jdy35i


wandb: Agent Starting Run: fukxykuh with config:
wandb: 	seed: 0


kan_mlp_periodic_ademamix on churn: 100%|██████████| 1/1 [00:00<00:00,  2.16it/s]


epoch,▁
full_train_time,▁
in_features,▁
lr,▁
num_params,▁
out_features,▁
seed,▁
test_acc,▁
test_loss,▁
test_time,▁
train_acc,▁


wandb: Agent Starting Run: gkuyhjsk with config:
wandb: 	seed: 1


...

epoch,▁
full_train_time,▁
in_features,▁
lr,▁
num_params,▁
out_features,▁
seed,▁
test_acc,▁
test_loss,▁
test_time,▁
train_acc,▁


wandb: Agent Starting Run: 7wb8l5be with config:
wandb: 	seed: 2
wandb: Ctrl + C detected. Stopping sweep.


KeyboardInterrupt: 

kan_mlp_periodic_ademamix on churn: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s]


epoch,▁
full_train_time,▁
in_features,▁
lr,▁
num_params,▁
out_features,▁
seed,▁
test_acc,▁
test_loss,▁
test_time,▁
train_acc,▁
